In [1]:
import numpy as np
import pandas as pd
import mne
from sklearn.preprocessing import StandardScaler
from spyeeg.models.TRF import TRFEstimator
import pickle

In [2]:
# thanks @ Pierre Guilleminot
def trf_spyeeg(regs, data, tmin, tmax, fs, alpha, channels):

    #:param regs: ndarray,  shape (n_times, n_reg)
    #:param data: ndarray,  shape (n_times, n_chan)
    #:param tmin: time-window min (in seconds)
    #:param tmax: time-window max (in seconds)
    #:param fs: frequency sampling of data and regressors
    #:param alpha: can be an int or a list
    #:param channels: list of channels names
    #:return: - r² score for each channel, shape : (n_chan)
    #- kernels of TRF, shape (tw_length (in sample), n_reg, n_chan)
 
    trf = TRFEstimator(tmin=tmin, tmax=tmax, srate=fs, alpha=alpha)
    #shape(n_folds, n_channels, n_alpha)
    scores_ = trf.xval_eval(regs, data, segment_length=None, fit_mode='direct', verbose = False)
    #shape (n_chan, n_alpha)
    scores_ = np.mean(scores_, axis=0)
    #shape (tw, n_reg, n_chan, n_alpha)
    kernels_ = trf.get_coef()
    ix_best_alpha = np.argmax(scores_, axis=1)
    #shape (n_chan)
    scores = np.zeros(len(channels))
    #shape (tw, n_reg, n_chan)
    kernels = np.zeros((kernels_.shape[0], kernels_.shape[1], kernels_.shape[2]))
    for i_chan in range(len(channels)):
        scores[i_chan] = scores_[i_chan, ix_best_alpha[i_chan]]
        kernels[:,:,i_chan] = kernels_[:,:,i_chan, ix_best_alpha[i_chan]]

    return scores_**2,kernels_
